In [ ]:
from fastai2.basics import *
from fastai2.callback.all import *
from fastai2.text.all import *

In [ ]:
# all_slow

# Integration test on Wikitext-2

> Training a Language Model on WT2

## Data

In [ ]:
path = untar_data(URLs.WIKITEXT_TINY)

The dataset comes with the articles in two csv files, so we read it and concatenate them in one dataframe.

In [ ]:
df_train = pd.read_csv(path/'train.csv', header=None)
df_valid = pd.read_csv(path/'test.csv', header=None)
df_all = pd.concat([df_train, df_valid])

In [ ]:
df_all.head()

,0
0,"\n = 2013 – 14 York City F.C. season = \n \n The 2013 – 14 season was the <unk> season of competitive association football and 77th season in the Football League played by York City Football Club , a professional football club based in York , North Yorkshire , England . Their 17th @-@ place finish in 2012 – 13 meant it was their second consecutive season in League Two . The season ran from 1 July 2013 to 30 June 2014 . \n Nigel Worthington , starting his first full season as York manager , made eight permanent summer signings . By the turn of the year York were only above the relegation z..."
1,"\n = Big Boy ( song ) = \n \n "" Big Boy "" <unk> "" I 'm A Big Boy Now "" was the first single ever recorded by the Jackson 5 , which was released by Steeltown Records in January 1968 . The group played instruments on many of their Steeltown compositions , including "" Big Boy "" . The song was neither a critical nor commercial success , but the Jackson family were delighted with the outcome nonetheless . \n The Jackson 5 would release a second single with Steeltown Records before moving to Motown Records . The group 's recordings at Steeltown Records were thought to be lost , but they were re..."
2,"\n = The Remix ( Lady Gaga album ) = \n \n The Remix is a remix album by American recording artist Lady Gaga . Released in Japan on March 3 , 2010 , it contains remixes of the songs from her first studio album , The Fame ( 2008 ) , and her third extended play , The Fame Monster ( 2009 ) . A revised version of the track list was prepared for release in additional markets , beginning with Mexico on May 3 , 2010 . A number of recording artists have produced the songs , including Pet Shop Boys , Passion Pit and The Sound of Arrows . The remixed versions feature both uptempo and <unk> composit..."
3,"\n = New Year 's Eve ( Up All Night ) = \n \n "" New Year 's Eve "" is the twelfth episode of the first season of the American comedy television series Up All Night . The episode originally aired on NBC in the United States on January 12 , 2012 . It was written by Erica <unk> and was directed by Beth McCarthy @-@ Miller . The episode also featured a guest appearance from Jason Lee as Chris and Reagan 's neighbor and Ava 's boyfriend , Kevin . \n During Reagan ( Christina Applegate ) and Chris 's ( Will <unk> ) first New Year 's Eve game night , Reagan 's competitiveness comes out causing Ch..."
4,"\n = Geopyxis carbonaria = \n \n Geopyxis carbonaria is a species of fungus in the genus Geopyxis , family <unk> . First described to science in 1805 , and given its current name in 1889 , the species is commonly known as the charcoal loving elf @-@ cup , dwarf <unk> cup , <unk> <unk> cup , or pixie cup . The small , <unk> @-@ shaped fruitbodies of the fungus are reddish @-@ brown with a whitish fringe and measure up to 2 cm ( 0 @.@ 8 in ) across . They have a short , tapered stalk . Fruitbodies are commonly found on soil where brush has recently been burned , sometimes in great numbers ...."


We could tokenize it based on spaces to compare (as is usually done) but here we'll use the standard fastai tokenizer.

In [ ]:
df_tok,count = tokenize_df(df_all, [0])
df_tok.head()

,text
0,"[xxbos, =, 2013, –, 14, xxmaj, york, xxmaj, city, xxup, f.c, ., season, =, \n▁\n▁, xxmaj, the, 2013, –, 14, season, was, the, xxunk, season, of, competitive, association, football, and, 77th, season, in, the, xxmaj, football, xxmaj, league, played, by, xxmaj, york, xxmaj, city, xxmaj, football, xxmaj, club, ,, a, professional, football, club, based, in, xxmaj, york, ,, xxmaj, north, xxmaj, yorkshire, ,, xxmaj, england, ., xxmaj, their, 17th, -, place, finish, in, 2012, –, 13, meant, it, was, their, second, consecutive, season, in, xxmaj, league, xxmaj, two, ., xxmaj, the, season, ran, from..."
1,"[xxbos, =, xxmaj, big, xxmaj, boy, (, song, ), =, \n▁\n▁, "", xxmaj, big, xxmaj, boy, "", xxunk, "", i, ', m, a, xxmaj, big, xxmaj, boy, xxmaj, now, "", was, the, first, single, ever, recorded, by, the, xxmaj, jackson, 5, ,, which, was, released, by, xxmaj, steeltown, xxmaj, records, in, xxmaj, january, 1968, ., xxmaj, the, group, played, instruments, on, many, of, their, xxmaj, steeltown, compositions, ,, including, "", xxmaj, big, xxmaj, boy, "", ., xxmaj, the, song, was, neither, a, critical, nor, commercial, success, ,, but, the, xxmaj, jackson, family, were, delighted, with, the, outcome, n..."
2,"[xxbos, =, xxmaj, the, xxmaj, remix, (, xxmaj, lady, xxmaj, gaga, album, ), =, \n▁\n▁, xxmaj, the, xxmaj, remix, is, a, remix, album, by, xxmaj, american, recording, artist, xxmaj, lady, xxmaj, gaga, ., xxmaj, released, in, xxmaj, japan, on, xxmaj, march, 3, ,, 2010, ,, it, contains, remixes, of, the, songs, from, her, first, studio, album, ,, xxmaj, the, xxmaj, fame, (, 2008, ), ,, and, her, third, extended, play, ,, xxmaj, the, xxmaj, fame, xxmaj, monster, (, 2009, ), ., a, revised, version, of, the, track, list, was, prepared, for, release, in, additional, markets, ,, beginning, with, x..."
3,"[xxbos, =, xxmaj, new, xxmaj, year, 's, xxmaj, eve, (, xxmaj, up, xxmaj, all, xxmaj, night, ), =, \n▁\n▁, "", xxmaj, new, xxmaj, year, 's, xxmaj, eve, "", is, the, twelfth, episode, of, the, first, season, of, the, xxmaj, american, comedy, television, series, xxmaj, up, xxmaj, all, xxmaj, night, ., xxmaj, the, episode, originally, aired, on, xxup, nbc, in, the, xxmaj, united, xxmaj, states, on, xxmaj, january, 12, ,, 2012, ., xxmaj, it, was, written, by, xxmaj, erica, xxunk, and, was, directed, by, xxmaj, beth, mccarthy, -, miller, ., xxmaj, the, episode, also, featured, a, guest, appearance..."
4,"[xxbos, =, xxmaj, geopyxis, carbonaria, =, \n▁\n▁, xxmaj, geopyxis, carbonaria, is, a, species, of, fungus, in, the, genus, xxmaj, geopyxis, ,, family, xxunk, ., xxmaj, first, described, to, science, in, 1805, ,, and, given, its, current, name, in, 1889, ,, the, species, is, commonly, known, as, the, charcoal, loving, elf, -, cup, ,, dwarf, xxunk, cup, ,, xxunk, xxunk, cup, ,, or, pixie, cup, ., xxmaj, the, small, ,, xxunk, -, shaped, fruitbodies, of, the, fungus, are, reddish, -, brown, with, a, whitish, fringe, and, measure, up, to, 2, cm, (, 0.8, in, ), across, ., xxmaj, they, have, a, ..."


The tokenize function returns a new dataframe with the tokenized texts and a counter with the frequency of each word. We use that counter to create a vocab, then a `Numericalize` transform.

In [ ]:
vocab = make_vocab(count)
tfm = Numericalize(make_vocab(count))

In [ ]:
splits = [list(range_of(df_train)), list(range(len(df_train), len(df_all)))]
dsrc = DataSource(df_tok, [[attrgetter("text"), tfm]], splits=splits, dl_type=LMDataLoader)

In [ ]:
bs,sl = 104,72
dbch = dsrc.databunch(bs=bs, seq_len=sl, after_batch=Cuda)

In [ ]:
dbch.show_batch()

,text,text_
0,"xxbos xxmaj naturally occurring cadmium is composed of 8 isotopes . xxmaj two of them are radioactive , and three are expected to decay but have not done so under laboratory conditions . xxmaj the two natural radioactive isotopes are xxunk ( beta decay , half - life is 7.7 × 1015 years ) and xxunk ( two - neutrino double beta decay , half - life is 2.9 × xxunk years","xxmaj naturally occurring cadmium is composed of 8 isotopes . xxmaj two of them are radioactive , and three are expected to decay but have not done so under laboratory conditions . xxmaj the two natural radioactive isotopes are xxunk ( beta decay , half - life is 7.7 × 1015 years ) and xxunk ( two - neutrino double beta decay , half - life is 2.9 × xxunk years )"
1,"de xxmaj janeiro xxunk xxunk wheel after the countdown to the beginning of 2009 . xxmaj the structure erected on xxmaj copacabana beach to promote the candidature was 36 m ( 118 ft 1 in ) high , weighed 80 tonnes ( 180 @,@ xxrep 3 0 lb ) and had 24 xxunk for 144 people . \n▁ xxmaj rio de xxmaj janeiro previously hosted major sporting , business and cultural events","xxmaj janeiro xxunk xxunk wheel after the countdown to the beginning of 2009 . xxmaj the structure erected on xxmaj copacabana beach to promote the candidature was 36 m ( 118 ft 1 in ) high , weighed 80 tonnes ( 180 @,@ xxrep 3 0 lb ) and had 24 xxunk for 144 people . \n▁ xxmaj rio de xxmaj janeiro previously hosted major sporting , business and cultural events ."
2,"well as how the player sometimes has to resort to a trial - and - error method due to the games only accepting specific pieces of evidence , and how testimony statements sometimes need to be pressed in a specific order . xxmaj some reviewers have criticized the lack of changes to the gameplay and presentation throughout the series , while some have said that fans of the series would not have","as how the player sometimes has to resort to a trial - and - error method due to the games only accepting specific pieces of evidence , and how testimony statements sometimes need to be pressed in a specific order . xxmaj some reviewers have criticized the lack of changes to the gameplay and presentation throughout the series , while some have said that fans of the series would not have a"
3,"gemma xxmaj constantine , xxmaj angie xxunk and xxmaj chas xxmaj chandler , a plan to reverse the problem is made , as tension builds among the soldiers now surrounding the city . xxmaj the soldiers xxunk listen to a xxmaj world xxmaj cup match between xxmaj england and xxmaj portugal on the radio . xxmaj when xxmaj england loses the match , it seems all is lost , but the expected","xxmaj constantine , xxmaj angie xxunk and xxmaj chas xxmaj chandler , a plan to reverse the problem is made , as tension builds among the soldiers now surrounding the city . xxmaj the soldiers xxunk listen to a xxmaj world xxmaj cup match between xxmaj england and xxmaj portugal on the radio . xxmaj when xxmaj england loses the match , it seems all is lost , but the expected psychic"
4,"xxmaj confederacy and the xxunk conquest of xxmaj ireland ) are estimated to include 20 @,@ xxrep 3 0 battlefield casualties . 200 @,@ xxrep 3 0 civilians are estimated to have died as a result of a combination of war - related famine , displacement , guerrilla activity and pestilence over the duration of the war . a further 50 @,@ xxrep 3 0 were sent into indentured servitude in the","confederacy and the xxunk conquest of xxmaj ireland ) are estimated to include 20 @,@ xxrep 3 0 battlefield casualties . 200 @,@ xxrep 3 0 civilians are estimated to have died as a result of a combination of war - related famine , displacement , guerrilla activity and pestilence over the duration of the war . a further 50 @,@ xxrep 3 0 were sent into indentured servitude in the xxmaj"
5,"heinrich xxmaj richter : "" i look at my things for what they are , as xxunk xxunk [ toys ] , as xxunk xxunk [ worthless and useless things ] whose value is 

## Model

In [ ]:
config = awd_lstm_lm_config.copy()
config.update({'input_p': 0.6, 'output_p': 0.4, 'weight_p': 0.5, 'embed_p': 0.1, 'hidden_p': 0.2})
model = get_language_model(AWD_LSTM, len(vocab), config=config)

In [ ]:
opt_func = partial(Adam, wd=0.1, eps=1e-7)
cb_funcs = [partial(MixedPrecision, clip=0.1), partial(RNNTrainer, alpha=2, beta=1)]

In [ ]:
learn = Learner(dbch, model, loss_func=CrossEntropyLossFlat(), opt_func=opt_func, cb_funcs=cb_funcs, metrics=[accuracy, Perplexity()])

In [ ]:
learn.fit_one_cycle(1, 5e-3, moms=(0.8,0.7,0.8), div=10)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,5.541510,5.050310,0.241224,156.070786,01:05


Full training

In [ ]:
#learn.fit_one_cycle(90, 5e-3, moms=(0.8,0.7,0.8), div=10)